# Calculate and plot the distribution of "cold pools"

In [1]:
import xarray as xr
import proplot as plot
import numpy as np
import glob

In [2]:
# exp = 'rcemip-small_v2'
exp = 'rcemip-large'

# cases = ['nz_32','nz_64','nz_128','nz_256']
cases = ['nz_32','nz_64','nz_128']

fromday = 0
to_day = 30

# where are files located
fpath = 

# colormap
cmap =  np.asarray([
                        [211, 62, 67, 255],
                        [248, 196, 98, 255],
                        [152, 193, 217, 255],
                        [64, 72, 119, 255]])/255

# where to save figure
figdir = 

In [3]:
# Load data and calculate virtual temperature for lowest model level
# Tv = T(1 + rv/ ε)/(1 + rv)
epsilon = 0.622

Tv = []

for case in cases:
    print(case)
    files = np.sort(glob.glob(fpath + case + '/OUT_3D/TABS/*.nc'))
    T = xr.open_mfdataset(files).isel(z=0).TABS.sel(time=slice(fromday,to_day))
    print('T loaded')
    
    files = np.sort(glob.glob(fpath + case + '/OUT_3D/QV/*.nc'))
    rv = xr.open_mfdataset(files).isel(z=0).QV.sel(time=slice(fromday,to_day)) / 1e3 # convert to kg/kg
    print('q loaded')
    
    Tv.append((T * (1 + (rv / epsilon)) / (1 + rv)).load())
    print('Tv loaded')
    
    del T
    del rv

nz_32
T loaded
q loaded
Tv loaded
nz_64
T loaded
q loaded
Tv loaded
nz_128
T loaded
q loaded
Tv loaded


In [ ]:
# At each time step, calculate the virtual temperature anomaly from the domain mean

Tv_anom = []

for icase in range(len(cases)):
    Tv_mean = Tv[icase].mean(dim=('x','y'))
    Tv_anom.append(Tv[icase] - Tv_mean)
    
del Tv

In [ ]:
# Calculate the distribution of virtual temperature anomlies 

bins = np.linspace(-5,0,num=50)
bincenter = bins[:-1] + np.diff(bins)[0]/2

Tv_hist = []

for icase in range(len(cases)):
    hist,_ = np.histogram(np.ravel(Tv_anom[icase].values), bins)
    Tv_hist.append(hist)

In [ ]:
fig, ax = plot.subplots()

for icase,case in enumerate(cases):
    ax.plot(bincenter, Tv_hist[icase]/sum(Tv_hist[icase]),
            color=cmap[icase],label=case)
    
ax.format(yscale='log',ylabel='Frequency',
          xlabel='Virtual Temperature Anomaly (K)',
          xlim=(-5,0))
ax.legend(ncol=1)
fig.savefig(figdir + exp + '_Tv_anom_distr_d' + str(fromday) + '-' + str(to_day) + '.png',dpi=300)